## This is my Toronto Peer Review Assginment

### Setting up my imports and let's pull the webpage so that we can scrape it  


In [1]:

import bs4 as bs
import urllib.request
import pandas as pd

source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()



### Let's use BeautifulSoup as suggested in the instructions for this assignment

In [2]:
soup = bs.BeautifulSoup(source,'lxml')

### Parse the data to extract the table data

In [3]:
#import re

table2 = soup.find('table',{"class":"wikitable sortable"})

table_rows2 = table2.find_all('tr')

l = []
for tr in table_rows2:
    td = tr.find_all('td')
    row2 = [tr.text for tr in td]
   
    l.append(row2)

del(l[0]) 


### Let's put in a df and remove the new line from the Neighbourhood entries

In [4]:
df2 = pd.DataFrame(l, columns=["Postal Code","Borough","Neighbourhood"])
df2['Neighbourhood'] = df2.Neighbourhood.str.replace(r'\n','')
df2.head(10)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


### Let's find where we have __"Not assigned"__ in Borough, but with __"Not assigned"__ in Neighborhood

In [5]:
inLoc = int(df2.query("Neighbourhood == 'Not assigned' and Borough != 'Not assigned'").index[0])

inLoc

8

### Adding Borough entry to Neighborhood per instructions when Borough has an entry and Neighborhood as Not assigned

In [6]:

df2.loc[inLoc][2] = df2.loc[inLoc][1]
df2.head(10)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
9,M8A,Not assigned,Not assigned


### Didn't want to use a for loop since I was able to find one entry that meets the condition, but just in case, let's run it again

In [7]:
df2.query("Neighbourhood == 'Not assigned' and Borough != 'Not assigned'")

# no other entry with the above condition, GREAT! Let's continue...

,Postal Code,Borough,Neighbourhood


### query returned zero. Let us move on... removing the remaining rows with entries where we found __"Not assigned"__

In [8]:
to_drop = {'Not assigned'}
df2 = df2.loc[~df2['Borough'].isin(to_drop) & ~df2['Neighbourhood'].isin(to_drop)] 
df2.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


### Resetting index number

In [9]:
df2 = df2.reset_index(drop=True)
df2.head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### Combining with groupby to Neighbourhood with Borough

In [10]:
df2 = df2.groupby("Postal Code").agg(lambda x: ', '.join(set(x))).reset_index()

In [11]:
df2.head(10)


,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Kennedy Park, Ionview"
7,M1L,Scarborough,"Clairlea, Oakridge, Golden Mile"
8,M1M,Scarborough,"Scarborough Village West, Cliffside, Cliffcrest"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### Per instructions, here is the .shape to my dataframe

In [12]:
df2.shape

(103, 3)

### Importing the Geospatial_data.csv file containing the latitude and longitude for this assignment's dataframe

In [13]:
!wget -O Geospatial_Coordinates.csv http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv

--2018-10-15 19:26:19--  http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv [following]
--2018-10-15 19:26:19--  https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2018-10-15 19:26:19--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.25.197
Connecting to ibm.box.com (ibm.box.com)|107.152.25.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5

## Load Data From CSV File

In [14]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Part 2 of the assignment merging the latitude and longitude

In [15]:
new_df = pd.merge(df2, df_geo[['Postal Code','Latitude','Longitude']],on='Postal Code')
new_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
